In this lab, you will create a custom CNN model for the CIFAR10 data set.

Helpful references:

https://www.youtube.com/watch?reload=9&v=eBbEDRsCmv4

In [ ]:
import tensorflow as tf
import time
import os
import numpy as np

Create two functions that will be reused in your custom model function.

_conv: convolution block to include conv, relu, and max pool.

_dense: dense block with relu.  Since last layer is logits, make relu switchable on/off.

(optional) for both blocks, report histogram for W, b, and activations.

In [ ]:
# Convolution Block

def _conv(x,kernel,name,log=False):
    with tf.variable_scope(name):
        #
        # code here
        #
        if log==True:
            #
            # code here
            #
        return # code here

# Dense Block

def _dense(x,size_in,size_out,name,relu=False,log=False):
    with tf.variable_scope(name):
        #
        # code here
        #
        if relu==True:
            #
            # code here
            #
        if log==True:
            #
            # code here
            #
        return activation

Create custom model function.

INFERENCE MODEL

Recommended architecture:
- conv1:   kernel = 5x5x128
- conv2:   kernel = 5x5x128
- conv3:   kernel = 3x3x256
- conv4:   kernel = 3x3x512
- dense:   hidden_units (tunable hyperparam)
- dropout: only for training



In [ ]:
def cnnmodel_fn(features, labels, mode, params):
    
    #### 1 INFERNCE MODEL
    
        #
        # code here
        #
    if mode==tf.estimator.ModeKeys.TRAIN:
        #
        # code here for dropout
        #
    logits = # code here
 
    #### 2 CALCULATIONS AND METRICS
    
    predictions = {
        # code here
    }
    export_outputs = {'predictions': # code here}
    if (mode==tf.estimator.ModeKeys.TRAIN or mode==tf.estimator.ModeKeys.EVAL):
        loss = # code here
        accuracy = tf.metrics.accuracy(# code here)
        metrics = {'accuracy':accuracy}
        
    #### 3 MODE = PREDICT
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(# code here)

    #### 4 MODE = TRAIN

    if mode == tf.estimator.ModeKeys.TRAIN:
        learning_rate = tf.train.exponential_decay(# code here)
        optimizer = # code here
        train_op = # code here
        tf.summary.scalar('learning_rate', learning_rate)
        tf.summary.scalar('accuracy',accuracy[1])
        return tf.estimator.EstimatorSpec(# code here)
    
    #### 5 MODE = EVAL
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(# code here)

Create function to deserialize tfrecords files.

Feature dictionary {idx, label, image}

- idx:   tf.int64
- label: tf.int64
- image: tf.string
    

In [ ]:
def parse_tfrecord(example):
    #
    # code here
    #
    return # code here


Create two helper functions:

image_scaling - applied always

distort - applied only in training.  Resize, crop, flip randomly.

In [ ]:
def image_scaling(x):
    return # code here

def distort(x):
    #
    # code here
    #
    return x

Create input function using tf.data

In [ ]:
def dataset_input_fn(params):
    #
    # code here
    #
    return dataset

In [ ]:
model_params  = {'drop_out'      : 0.2,
                 'dense_units'   : 1024,
                 'learning_rate' : 1e-3,
                 'log'           : True
                }

In [ ]:
config = tf.estimator.RunConfig(save_checkpoints_secs = 300,keep_checkpoint_max = 5)

In [ ]:
#Set model_fn
model_fn = cnnmodel_fn

In [ ]:
name = 'cnn_model/cnn_model_'
name = name + 'dense' + str(model_params['dense_units']) + '_'
name = name + 'drop' + str(model_params['drop_out']) + '_'
name = name + 'lr' + str(model_params['learning_rate']) + '_'
name = name + time.strftime("%Y%m%d%H%M%S")
model_dir  = os.path.join('./',name)

print(model_dir)

In [ ]:
estimator = tf.estimator.Estimator(
    model_fn=model_fn,model_dir=model_dir,params=model_params,config=config)

In [ ]:
train_files = !ls cifar10_data_00*.tfrecords
val_files   = !ls cifar10_data_01*.tfrecords

train_params = {'filenames'    : train_files,
                'mode'         : tf.estimator.ModeKeys.TRAIN,
                'threads'      : 16,
                'shuffle_buff' : 100000,
                'batch'        : 100
               }

eval_params  = {'filenames'    : val_files,
                'mode'         : tf.estimator.ModeKeys.EVAL,
                'threads'      : 8,
                'batch'        : 200
               }

train_spec = tf.estimator.TrainSpec(input_fn=lambda: dataset_input_fn(train_params),max_steps=20000)
eval_spec  = tf.estimator.EvalSpec(input_fn=lambda: dataset_input_fn(eval_params),steps=10,throttle_secs=60)


In [ ]:
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)